
# Faces recognition example using RESNET


The dataset used in this example is a preprocessed excerpt of the
"Labeled Faces in the Wild", aka LFW_:

  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (233MB)


Let's check how the popular residual network (aka RESNET) performs with a subset of images of the above dataset.
We shall use below off-the-shelve implementation

[DLIB RESNET](http://blog.dlib.net/2017/02/high-quality-face-recognition-with-deep.html)

Let's first load necessary modules and data

In [9]:
from time import time
import logging


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.svm import SVC
import dlib
import numpy as np


print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')


# #############################################################################
# Download the data, if not already on disk and load it as numpy arrays

lfw_people = fetch_lfw_people(min_faces_per_person=30, resize=0.5,color=True)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w, _ = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
Images = lfw_people.images
n_features = Images.shape[1]

Automatically created module for IPython interactive environment


Now load models

In [10]:
sp = dlib.shape_predictor('models/shape_predictor_5_face_landmarks.dat')

facerec = dlib.face_recognition_model_v1('models/dlib_face_recognition_resnet_model_v1.dat')

Compute face feature descriptors

In [11]:
X=[]

for i in Images:
    shape = sp(i.astype(np.uint8), dlib.rectangle(0,0,w,h))
    face_descriptor = facerec.compute_face_descriptor(i.astype(np.uint8), shape)
    X.append(face_descriptor)

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

Create train and test data sets

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Finally train or classifier and make some predictions

In [13]:
print("Fitting the classifier to the training set")
t0 = time()
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)

clf = clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))

print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test)
print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred, target_names=target_names))

print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))

Fitting the classifier to the training set
done in 75.810s
Predicting people's names on the test set
done in 0.192s
                           precision    recall  f1-score   support

         Alejandro Toledo       0.88      1.00      0.93         7
             Alvaro Uribe       1.00      1.00      1.00         9
             Andre Agassi       1.00      1.00      1.00        10
             Ariel Sharon       1.00      1.00      1.00        19
    Arnold Schwarzenegger       1.00      1.00      1.00        11
             Colin Powell       1.00      1.00      1.00        55
            David Beckham       1.00      1.00      1.00         6
          Donald Rumsfeld       1.00      1.00      1.00        34
            George W Bush       0.99      1.00      1.00       140
        Gerhard Schroeder       1.00      0.97      0.98        29
  Gloria Macapagal Arroyo       1.00      1.00      1.00        10
          Guillermo Coria       1.00      1.00      1.00        10
            

Voilà, we got 99% accuracy, pretty good! Particularly if compared with a more classical feature descriptor such as PCA, found in scikit library [face recognition demo](https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html#sphx-glr-auto-examples-applications-plot-face-recognition-py). The example on scikit if run with `fetch_lfw_people(min_faces_per_person=30, ...)` will give an accuracy of around 65%.